In [61]:
import os

data_dir = "/home/ubuntu/data/lastfm/lastfm-dataset-360K"
os.environ["DATA_DIR"] = data_dir

In [149]:
def lastfm_to_libfm(infile: str, outfile: str, outfile2: str) -> None:
    with open(os.path.join(data_dir, infile), "r") as rf:
        with open(os.path.join(data_dir, outfile), "w") as wf:
            with open(os.path.join(data_dir, outfile2), "w") as wf2:
                for line in rf:
                    arr = line.split("\t")
                    target = 0
                    if int(arr[2]) >= 100:
                        target = 1
                        wf2.write("{}".format(line))
                    wf.write("{}\t{}\t{}\n".format(arr[0], arr[1], target))

lastfm_to_libfm("train/lastfm.train.0", "libfm_train", "libfm.train.0")
lastfm_to_libfm("lastfm.test.0", "libfm_test", "libfm.test.0")

In [127]:
!head $DATA_DIR/libfm_train

1	15531	1
1	63469	1
1	15968	1
1	48969	1
1	36493	1
1	26332	1
1	8804	1
1	42630	1
1	64575	1
1	32115	1


In [128]:
!triple_format_to_libfm \
    -in $DATA_DIR/libfm_train \
    -target 2 \
    -separator "\t"

transforming file /home/ubuntu/data/lastfm/lastfm-dataset-360K/libfm_train to /home/ubuntu/data/lastfm/lastfm-dataset-360K/libfm_train.libfm...


In [129]:
!triple_format_to_libfm \
    -in $DATA_DIR/libfm_test \
    -target 2 \
    -separator "\t"

transforming file /home/ubuntu/data/lastfm/lastfm-dataset-360K/libfm_test to /home/ubuntu/data/lastfm/lastfm-dataset-360K/libfm_test.libfm...


In [130]:
!head $DATA_DIR/libfm_train.libfm

1 0:1 1:1
1 0:1 2:1
1 0:1 3:1
1 0:1 4:1
1 0:1 5:1
1 0:1 6:1
1 0:1 7:1
1 0:1 8:1
1 0:1 9:1
1 0:1 10:1


In [153]:
!libFM \
    -train $DATA_DIR/libfm_train.libfm \
    -test $DATA_DIR/libfm_test.libfm \
    -task c \
    -out $DATA_DIR/recs.libfm \
    -iter 1

----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...	
has x = 0
has xt = 1
num_rows=799785	num_values=1599570	num_features=81495	min_target=0	max_target=1
Loading test... 	
has x = 0
has xt = 1
num_rows=200000	num_values=400000	num_features=52379	min_target=0	max_target=1
#relations: 0
Loading meta data...	
#Iter=  0	Train=0.817694	Test=0.501695	Test(ll)=0.305484


In [154]:
!head $DATA_DIR/recs.libfm

0.59476
0.584219
0.556064
0.537367
0.648803
0.541951
0.531945
0.60628
0.586736
0.605348


In [155]:
!awk -F '\t' 'BEGIN {OFS = FS} NF{NF-=1};1' <$DATA_DIR/libfm_test >$DATA_DIR/libfm_test.no_target

In [156]:
!head $DATA_DIR/libfm_test.no_target

8631	55334
7975	28244
15296	35494
20417	63251
16563	61228
11839	27930
5150	64176
3475	12769
12350	38225
7890	43761


In [157]:
!paste $DATA_DIR/libfm_test.no_target $DATA_DIR/recs.libfm > $DATA_DIR/recs.libfm.pasted

In [158]:
!head $DATA_DIR/recs.libfm.pasted

8631	55334	0.59476
7975	28244	0.584219
15296	35494	0.556064
20417	63251	0.537367
16563	61228	0.648803
11839	27930	0.541951
5150	64176	0.531945
3475	12769	0.60628
12350	38225	0.586736
7890	43761	0.605348


In [159]:
with open(os.path.join(data_dir, "recs.libfm.pasted"), "r") as rf:
    with open(os.path.join(data_dir, "recs2/libfm.test.0.recs.tsv"), "w") as wf:
        for line in rf:
            arr = line.split("\t")
            if float(arr[2]) > 0.5:
                target = 1
                wf.write("{}".format(line))

In [160]:
# используем mrec в качестве метрики качества
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/libfm.test.0 \
    --recsdir $DATA_DIR/recs2

[2017-11-19 15:15:51,442] INFO: processing /home/ubuntu/data/lastfm/lastfm-dataset-360K/libfm.test.0...
None
mrr            0.6174 +/- 0.0000
prec@5         0.1235 +/- 0.0000
prec@10        0.0617 +/- 0.0000
prec@15        0.0412 +/- 0.0000
prec@20        0.0309 +/- 0.0000
